In [1]:
import pandas as pd
import re
import sqlite3

In [2]:
bisnis = pd.read_csv('bisnis_news.csv')
bisnis['source'] = 'bisnis'

cnbc = pd.read_csv('cnbc_news.csv')
cnbc['source'] = 'cnbc'

idx1 = pd.read_csv('idxchannel_news.csv')
idx2 = pd.read_csv('idxchannel_news2.csv')
idx = pd.concat([idx1, idx2], ignore_index=True)
idx = idx[idx['category']=='market-news'].drop(columns=['category'], axis=1)
idx['source'] = 'idx'

investor = pd.read_csv('investor_id_news.csv')
investor['source'] = 'investorid'

iqplus = pd.read_csv('iqplus_news.csv')
iqplus['source'] = 'iqplus'

kontan = pd.read_csv('kontan_news.csv')
kontan['source'] = 'kontan'

In [3]:
df = pd.concat([bisnis, cnbc, idx, investor, iqplus, kontan], ignore_index=True)

In [4]:
df.columns

Index(['published', 'link', 'title', 'content', 'source'], dtype='object')

In [5]:
df.isna().sum()

published    0
link         0
title        0
content      1
source       0
dtype: int64

In [6]:
len(df)

1102

In [7]:
df = df.drop_duplicates(subset=['link'])

In [8]:
len(df)

1101

In [9]:
df.isna().sum()

published    0
link         0
title        0
content      1
source       0
dtype: int64

In [10]:
df['source'].value_counts()

source
kontan        447
iqplus        300
bisnis        223
cnbc          100
investorid     30
idx             1
Name: count, dtype: int64

In [11]:
df[df['content'].isna()]

,published,link,title,content,source
986,2025-08-07,https://investasi.kontan.co.id/news/ihsg-mengu...,"IHSG Menguat 0,49% di Sesi Pertama Hari Ini (7...",NaN,kontan


In [12]:
df = df.reset_index(drop=True)

In [13]:
def clean_invisible_spaces(text):
    return re.sub(r'[\u00A0\u2000-\u200B\u202F\u205F\u3000]', ' ', text).strip()

PHRASES_PATTERN = re.compile(
    r'Jakarta, CNBC Indonesia|'
    r'IDXChannel—|KONTAN\.CO\.ID JAKARTA\.?|'
    r'Bisnis\.com , JAKARTA|Bisnis\.com, JAKARTA|'
    r'JAKARTA, investor\.id|Pasardana\.id|KONTAN\.CO\.ID|'
    r'IQPlus,{0,}\s{0,}\(\d+/\d+\)',
    re.IGNORECASE
)
LEADING_PUNCT_PATTERN = re.compile(r'^[^\w\s]+')
DASH_PATTERN = re.compile(r'\s*[-–—]\s*')
MULTISPACE_PATTERN = re.compile(r'\s+')

def clean_text(text):
    if not isinstance(text, str):
        return text
    text = clean_invisible_spaces(text)
    text = PHRASES_PATTERN.sub(' ', text).strip()
    text = LEADING_PUNCT_PATTERN.sub('', text)
    text = DASH_PATTERN.sub(' ', text)
    text = MULTISPACE_PATTERN.sub(' ', text)
    return text.strip()

In [14]:
df['content'] = df['content'].apply(clean_text)

In [15]:
df[df['source']=='iqplus']['content'].sample(5)

565    PT BFI Finance Indonesia Tbk (BFIN) telah mene...
612    PT Bank Rakyat Indonesia (Persero) Tbk atau BR...
640    Sebagai bagian dari perannya dalam mendorong i...
410    Sani Handoko selaku Komisaris utama PT Multika...
516    PT Nusa Palapa Gemilang Tbk (NPGF) meraih penj...
Name: content, dtype: object

In [16]:
df['DW_LOAD_TS'] = pd.to_datetime('now')
df['DW_LOAD_TS'] = df['DW_LOAD_TS'].dt.tz_localize('Asia/Jakarta')
df['DW_MODIFY_TS'] = pd.to_datetime('now')
df['DW_MODIFY_TS'] = df['DW_MODIFY_TS'].dt.tz_localize('Asia/Jakarta')

In [17]:
db_path = "stock_news.db"
table_name = "news"
with sqlite3.connect(db_path) as conn:
    df.to_sql(table_name, conn, if_exists="replace", index=False, dtype={
        'content': 'TEXT'  # explicitly set to TEXT
    })

In [18]:
len(df)

1101